## 纯位置控制与位置速度控制对比

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 1. 仿真设置
dt = 0.001       # 控制周期 1ms (1000Hz)
total_time = 2.0 # 总时间 2秒
t = np.arange(0, total_time, dt)

# 2. 定义目标轨迹 (模拟上层规划给出的指令)
# 假设我们要让关节像摆锤一样摆动：sin(t)
theta_d = np.sin(5 * t)           # 目标位置 (来自位置 IK)
theta_dot_d = 5 * np.cos(5 * t)   # 目标速度 (来自速度 IK, J_inv * V_d)

# 3. 定义电机物理参数和PID参数
Inertia = 0.1   # 电机转动惯量
Kp = 100.0      # 位置增益
Kd = 5.0        # 速度/阻尼增益

# 4. 初始化两个系统的状态
# 系统A: 只有位置控制
theta_A = 0.0
theta_dot_A = 0.0
history_A = []

# 系统B: 位置 + 速度前馈 (Full Control)
theta_B = 0.0
theta_dot_B = 0.0
history_B = []

# 5. 开始仿真循环
for i in range(len(t)):
    # --- 系统 A (无速度 IK) ---
    # 控制律: tau = Kp * (误差) + Kd * (0 - 当前速度)
    # 这里的 0 意味着我们不知道目标速度，只能把它当做阻尼来用
    error_pos_A = theta_d[i] - theta_A
    error_vel_A = 0 - theta_dot_A  # 缺少 theta_dot_d !
    tau_A = Kp * error_pos_A + Kd * error_vel_A
    
    # 物理积分 (F=ma -> a=F/m)
    acc_A = tau_A / Inertia
    theta_dot_A += acc_A * dt
    theta_A += theta_dot_A * dt
    history_A.append(theta_A)

    # --- 系统 B (有速度 IK) ---
    # 控制律: tau = Kp * (误差) + Kd * (目标速度 - 当前速度)
    # 我们利用了逆速度运动学提供的 theta_dot_d
    error_pos_B = theta_d[i] - theta_B
    error_vel_B = theta_dot_d[i] - theta_dot_B # 这里的 theta_dot_d 极其关键
    tau_B = Kp * error_pos_B + Kd * error_vel_B
    
    # 物理积分
    acc_B = tau_B / Inertia
    theta_dot_B += acc_B * dt
    theta_B += theta_dot_B * dt
    history_B.append(theta_B)

# 6. 绘图结果
plt.figure(figsize=(10, 6))
plt.plot(t, theta_d, 'k--', linewidth=2, label='Target (Plan)')
plt.plot(t, history_A, 'r-', linewidth=1.5, label='System A (No Vel IK) - Lagging')
plt.plot(t, history_B, 'g-', linewidth=1.5, label='System B (With Vel IK) - Perfect')
plt.title('Ablation Study: Why do we need Inverse Velocity Kinematics?')
plt.xlabel('Time (s)')
plt.ylabel('Joint Angle (rad)')
plt.legend()
plt.grid(True)
plt.show()